# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [225]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [226]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [227]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [228]:
quantities = data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
quantities.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [229]:
quantities_pivot = pd.pivot_table(quantities, index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
quantities_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [230]:
distances = pd.DataFrame(1/(1 + squareform(pdist(quantities_pivot.T, 'euclidean'))), 
                         index=quantities_pivot.columns, columns=quantities_pivot.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [231]:
similarities = distances[33].sort_values(ascending=False)[1:]
some_values = []
for k, j in dict(similarities.head()).items():
    some_values.append(k)
some_values

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [232]:
data5 = quantities.loc[quantities['CustomerID'].isin(some_values)]
data5.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [233]:
data5_new = data5.groupby('ProductName').agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending=False)
data5_new.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [234]:
merged = data5_new.merge(quantities_pivot, on='ProductName')
merged = merged.loc[merged[33]==0].sort_values(['Quantity'],ascending=False)['Quantity'].head()
pd.DataFrame(merged)

,Quantity
ProductName,
Butter - Unsalted,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
Wine - Ej Gallo Sierra Valley,3
Beans - Wax,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [240]:
def recommendations(distance):
    empty = {}
    CustomerIDs = data['CustomerID'].sample(n=6, random_state=0)
                  #[33, 200, 264, 356, 412]
    
    distances = pd.DataFrame(1/(1 + squareform(pdist(quantities_pivot.T, distance))), 
                             index=quantities_pivot.columns, columns=quantities_pivot.columns)
    
    for i in CustomerIDs:
        similarities = distances[i].sort_values(ascending=False)[1:]
        some_values = []
        for k, j in dict(similarities.head()).items():
            some_values.append(k)
        data5 = quantities.loc[quantities['CustomerID'].isin(some_values)]
        data5_new = data5.groupby('ProductName').agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending=False)
        merged = data5_new.merge(quantities_pivot, on='ProductName')
        empty[i] = merged = merged.loc[merged[33]==0].sort_values(['Quantity'],ascending=False)[[i,'Quantity']].head().index

    return empty

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [241]:
pd.DataFrame.from_dict(recommendations('euclidean'))

,11102,84577,17472,47913,82597,44742
0,Flavouring - Orange,Bread Crumbs - Panko,Rice - Long Grain,Tomatoes Tear Drop,Table Cloth 81x81 White,"Mushroom - Trumpet, Dry"
1,Coffee - Hazelnut Cream,Milk - 1%,"Cheese - Boursin, Garlic / Herbs",Pickerel - Fillets,Milk - 2%,"Turnip - White, Organic"
2,Sardines,Cookie Chocolate Chip With,Rabbit - Whole,Cornflakes,"Cheese - Brie, Triple Creme",Squid U5 - Thailand
3,Banana - Leaves,Milk - 2%,"Yogurt - Blueberry, 175 Gr",Pants Custom Dry Clean,Muffin Chocolate Individual Wrap,Crush - Cream Soda
4,Cheese Cloth No 100,"Pepsi - Diet, 355 Ml",Shrimp - 31/40,Ecolab - Mikroklene 4/4 L,Sun - Dried Tomatoes,Cumin - Whole


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [244]:
distances = ['correlation','cityblock','cosine','jaccard', 'chebyshev', 'rogerstanimoto', 'sokalmichener']
for i in distances:
    display(pd.DataFrame.from_dict(recommendations(i)))

,11102,84577,17472,47913,82597,44742
0,Cookies - Assorted,Napkin White - Starched,"Bread - Roll, Soft White Round",Flour - Teff,Anchovy Paste - 56 G Tube,Wine - Toasted Head
1,"Chestnuts - Whole,canned","Sauce - Gravy, Au Jus, Mix",Beef - Inside Round,Cup - Translucent 7 Oz Clear,Puree - Passion Fruit,Turkey - Oven Roast Breast
2,Loquat,Sage - Ground,Truffle Cups - Brown,Beef - Striploin Aa,Broom - Corn,"Lemonade - Natural, 591 Ml"
3,Pie Filling - Cherry,Spinach - Baby,"Beans - Kidney, Red Dry",Banana Turning,Wine - Chardonnay South,"Turnip - White, Organic"
4,Veal - Inside,Beef - Texas Style Burger,Muffin Chocolate Individual Wrap,Cake - Cake Sheet Macaroon,Wine - Vineland Estate Semi - Dry,Veal - Osso Bucco


,11102,84577,17472,47913,82597,44742
0,Ice Cream Bar - Hageen Daz To,Wanton Wrap,Beans - Wax,Sauce - Rosee,Ice Cream Bar - Hageen Daz To,Wanton Wrap
1,Cake - Cake Sheet Macaroon,Beef - Texas Style Burger,Cod - Black Whole Fillet,Sausage - Liver,Juice - Lime,"Wine - Magnotta, Merlot Sr Vqa"
2,Olive - Spread Tapenade,Lime Cordial - Roses,Fond - Neutral,Ice Cream Bar - Oreo Cone,Pork - Kidney,Ice Cream Bar - Oreo Cone
3,Pork - Kidney,Isomalt,Soup Campbells - Italian Wedding,"Beets - Candy Cane, Organic",Juice - Apple Cider,Soup - Campbells Bean Medley
4,Juice - Apple Cider,"Wine - Magnotta, Merlot Sr Vqa","Cheese - Boursin, Garlic / Herbs",Soup - Campbells Bean Medley,Wine - Toasted Head,Scallops - 10/20


,11102,84577,17472,47913,82597,44742
0,Olives - Stuffed,Cookie Chocolate Chip With,"Bread - Roll, Soft White Round",Flour - Teff,Wine - Chardonnay South,Cornflakes
1,Cookies - Assorted,"Pepsi - Diet, 355 Ml",Beef - Inside Round,Cup - Translucent 7 Oz Clear,Wine - Vineland Estate Semi - Dry,Table Cloth 81x81 White
2,Loquat,Table Cloth 62x114 White,Truffle Cups - Brown,Beef - Striploin Aa,Anchovy Paste - 56 G Tube,"Ice - Clear, 300 Lb For Carving"
3,Initation Crab Meat,Spinach - Baby,"Beans - Kidney, Red Dry",Banana Turning,Bread Fig And Almond,Wine - Toasted Head
4,Bread - Rye,Bread Crumbs - Panko,Muffin Chocolate Individual Wrap,Cake - Cake Sheet Macaroon,Wine - Prosecco Valdobiaddene,"Tart Shells - Sweet, 4"


,11102,84577,17472,47913,82597,44742
0,Bay Leaf,Tia Maria,Shrimp - 31/40,Mussels - Cultivated,Curry Paste - Madras,Milk - 1%
1,Soup Knorr Chili With Beans,"Wine - Magnotta, Merlot Sr Vqa",Clam Nectar,Pants Custom Dry Clean,Lime Cordial - Roses,Ice Cream Bar - Oreo Cone
2,Chocolate - Compound Coating,Muffin Chocolate Individual Wrap,Cocoa Butter,Cookie Chocolate Chip With,Cup - Translucent 7 Oz Clear,Bread - Italian Corn Meal Poly
3,Garlic - Peeled,Tea - Herbal Sweet Dreams,Nantuket Peach Orange,"Tart Shells - Sweet, 4","Garlic - Primerba, Paste",Lettuce - California Mix
4,Banana Turning,"Chocolate - Semi Sweet, Calets",Bacardi Breezer - Tropical,Ketchup - Tomato,Tea - Earl Grey,Ecolab - Solid Fusion


,11102,84577,17472,47913,82597,44742
0,Juice - Orange,Juice - Apple Cider,"Oil - Shortening,liqud, Fry",Wine - Fume Blanc Fetzer,"Beer - Alexander Kieths, Pale Ale",Cornflakes
1,Watercress,Durian Fruit,Spinach - Baby,Skirt - 29 Foot,"Cheese - Brie,danish",Wasabi Powder
2,Knife Plastic - White,Shrimp - 31/40,Pastry - Butterscotch Baked,Ketchup - Tomato,Macaroons - Two Bite Choc,"Spoon - Soup, Plastic"
3,Beef - Texas Style Burger,Halibut - Steaks,Truffle Cups - Brown,Arizona - Green Tea,Cumin - Whole,Truffle Cups - Brown
4,Apricots - Halves,Hinge W Undercut,Sponge Cake Mix - Chocolate,Wine - Redchard Merritt,Wine - Blue Nun Qualitatswein,Milk - 1%


,11102,84577,17472,47913,82597,44742
0,Beef - Montreal Smoked Brisket,"Sauce - Gravy, Au Jus, Mix",Cookie Dough - Double,Soup - Campbells Tomato Ravioli,"Appetizer - Mini Egg Roll, Shrimp","Wine - White, Mosel Gold"
1,Pie Filling - Cherry,Bandage - Fexible 1x3,V8 - Berry Blend,Broom - Corn,Tomatoes Tear Drop,"Lemonade - Natural, 591 Ml"
2,Snapple Lemon Tea,Cheese - Taleggio D.o.p.,Cheese - Parmesan Cubes,Beef Ground Medium,Beef - Rib Eye Aaa,Cheese - Parmesan Cubes
3,Zucchini - Yellow,Sage - Ground,Duck - Breast,"Soup - Campbells, Lentil",Bread - Bistro White,Bandage - Fexible 1x3
4,"Beets - Candy Cane, Organic","Appetizer - Mini Egg Roll, Shrimp",Halibut - Steaks,"Mushrooms - Black, Dried",Beef - Texas Style Burger,Wine - Toasted Head


,11102,84577,17472,47913,82597,44742
0,Beef - Montreal Smoked Brisket,"Sauce - Gravy, Au Jus, Mix",Cookie Dough - Double,Soup - Campbells Tomato Ravioli,"Appetizer - Mini Egg Roll, Shrimp","Wine - White, Mosel Gold"
1,Pie Filling - Cherry,Bandage - Fexible 1x3,V8 - Berry Blend,Broom - Corn,Tomatoes Tear Drop,"Lemonade - Natural, 591 Ml"
2,Snapple Lemon Tea,Cheese - Taleggio D.o.p.,Cheese - Parmesan Cubes,Beef Ground Medium,Beef - Rib Eye Aaa,Cheese - Parmesan Cubes
3,Zucchini - Yellow,Sage - Ground,Duck - Breast,"Soup - Campbells, Lentil",Bread - Bistro White,Bandage - Fexible 1x3
4,"Beets - Candy Cane, Organic","Appetizer - Mini Egg Roll, Shrimp",Halibut - Steaks,"Mushrooms - Black, Dried",Beef - Texas Style Burger,Wine - Toasted Head
